In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_excel('scrapped_data.xls')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [3]:
data.head()

,rowid,date,commodity,total_value_sold,total_qty_sold,total_kg_sold,qty_available
0,1,20 August 2020,AMADUMBE,"R0.00MTD: R39,870.00",0MTD: 97,"0MTD: 1,940",2
1,2,20 August 2020,APPLES,"R1,205,932.00MTD: R22,664,221.00","13,799MTD: 261,296","157,462MTD: 3,163,863",91755
2,3,20 August 2020,ARTICHOKES,"R600.00MTD: R53,100.00",4MTD: 439,3MTD: 522,1
3,4,20 August 2020,ASPARAGUS,"R34,000.00MTD: R258,975.00",50MTD: 359,"250MTD: 1,795",8
4,5,20 August 2020,ATCHARA,"R0.00MTD: R1,351.20",0MTD: 23,0MTD: 65,207


In [4]:
float(data['total_value_sold'][0].split("R")[-1].replace(',',''))

39870.0

In [5]:
float(data['total_value_sold'][0].split("R")[-1].replace(',',''))

39870.0

In [6]:
float(data['total_value_sold'][1].split("R")[1].split("M")[0].replace(",",""))

1205932.0

In [7]:
def rand_value(df, column):
    df["MTD_total_value_sold_(R)"]=np.nan
    df["total_value_sold_(R)"]=np.nan
    for i in range(len(df[column])):
        df["MTD_total_value_sold_(R)"][i]=round(float(df[column][i].split("R")[-1].replace(',','')),2)
        df["total_value_sold_(R)"][i]=round(float(df[column][i].split("R")[1].split("M")[0].replace(",","")),2)
        
    df.drop(column,axis=1, inplace=True)
    return df
    

In [8]:
df1=rand_value(data,"total_value_sold")

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df1.head()

,rowid,date,commodity,total_qty_sold,total_kg_sold,qty_available,MTD_total_value_sold_(R),total_value_sold_(R)
0,1,20 August 2020,AMADUMBE,0MTD: 97,"0MTD: 1,940",2,39870.0,0.0
1,2,20 August 2020,APPLES,"13,799MTD: 261,296","157,462MTD: 3,163,863",91755,22664221.0,1205932.0
2,3,20 August 2020,ARTICHOKES,4MTD: 439,3MTD: 522,1,53100.0,600.0
3,4,20 August 2020,ASPARAGUS,50MTD: 359,"250MTD: 1,795",8,258975.0,34000.0
4,5,20 August 2020,ATCHARA,0MTD: 23,0MTD: 65,207,1351.2,0.0


In [10]:
int(data["total_qty_sold"][2].split("M")[0].replace(",",""))

4

In [11]:
def sold_qty_kg(df,quatity_sold, weight_sold):
    df["Total_quatity_sold"]=np.nan
    df["MTD_Total_quatity_sold"]=np.nan
    df["Total_kg_sold"]=np.nan
    df["MTD_total_kg_sold"]=np.nan
    for i in range(len(df[quatity_sold])):
        df["Total_quatity_sold"][i]=int(df[quatity_sold][i].split("M")[0].replace(",",""))
        df["MTD_Total_quatity_sold"][i]=int(df[quatity_sold][i].split(":")[-1].replace(',',''))
        df["Total_kg_sold"][i]=int(df[weight_sold][i].split("M")[0].replace(",",""))
        df["MTD_total_kg_sold"][i]=int(df[weight_sold][i].split(":")[-1].replace(',',''))
    df.drop([quatity_sold,weight_sold],axis=1, inplace=True)
    return df

In [12]:
df2= sold_qty_kg(df1, "total_qty_sold","total_kg_sold")

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launc

In [13]:
df2.head()

,rowid,date,commodity,qty_available,MTD_total_value_sold_(R),total_value_sold_(R),Total_quatity_sold,MTD_Total_quatity_sold,Total_kg_sold,MTD_total_kg_sold
0,1,20 August 2020,AMADUMBE,2,39870.0,0.0,0.0,97.0,0.0,1940.0
1,2,20 August 2020,APPLES,91755,22664221.0,1205932.0,13799.0,261296.0,157462.0,3163863.0
2,3,20 August 2020,ARTICHOKES,1,53100.0,600.0,4.0,439.0,3.0,522.0
3,4,20 August 2020,ASPARAGUS,8,258975.0,34000.0,50.0,359.0,250.0,1795.0
4,5,20 August 2020,ATCHARA,207,1351.2,0.0,0.0,23.0,0.0,65.0
